## 1- Library and Data Imports

In [1]:
import numpy as np
import pandas as pd
import time

# for Web Scraping
import requests
from bs4 import BeautifulSoup

# for text cleaning and preprocessing
import re
from nltk.corpus import stopwords
import string 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
docs_df = pd.read_json('../Data/husna.json')
docs_df.head()

,_id,title,url,publisher,published_at,crawled_at,summary,content,tags,article_type
0,633fdfd13ffae8229d05cb33,التربية: تحويل 42 مدرسة إلى نظام الفترتين واست...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,husna,2022-10-06 07:33:00,2022-10-07 08:14:09.717,,[أكدت أمين عام وزارة التربية والتعليم للشؤون...,"[التربية والتعليم, وزارة التربية والتعليم]",News
1,633fdfd13ffae8229d05cb34,تكريما للمعلمين زيادة منح أبناء المعلمين 550 م...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,husna,2022-10-05 11:46:00,2022-10-07 08:14:09.788,,[احتفلت وزارة التربية والتعليم بيوم المعلم ب...,"[مكرمة أبناء المعلمين, وزارة التربية والتعليم]",News
2,633fdfd13ffae8229d05cb35,الغاز الروسي يضع أوروبا خلال الشتاء المقبل أما...,https://husna.fm/%D9%85%D9%84%D9%81%D8%A7%D8%A...,husna,2022-08-22 07:17:00,2022-10-07 08:14:09.801,,[يشهد العالم أول أزمة طاقة عالمية حقيقية في ال...,"[الغاز الروسي, أوروبا, أزمة الطاقة]",News
3,633fdfd13ffae8229d05cb36,تفاصيل دوام المدارس بعد قرار تثبيت التوقيت الصيفي,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,husna,2022-10-06 05:29:00,2022-10-07 08:14:09.811,,[كشفت أمين عام وزارة التربية والتعليم للشؤون ا...,"[التربية والتعليم, التوقيت الصيفي, المدارس]",News
4,633fdfd13ffae8229d05cb37,العمل: عطلة المولد النبوي الشريف تشمل القطاع ا...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,husna,2022-10-06 07:46:00,2022-10-07 08:14:09.864,,[أكدت وزارة العمل أن العطل الرسمية تكون مأجو...,"[وزارة العمل, المولد النبوي]",News


## 2- Data Preparation and Cleaning

Cleaning Functions Applied:
- removing mentions
- removing punctuation
- removing Arabic diacritics (short vowels and other harakahs)
- removing elongation
- removing stopwords (which is available in NLTK corpus)

#### 2.1 preparing data for cleaning

In [14]:
docs_df = docs_df.drop(columns=['publisher', 'crawled_at'], axis=1)

In [27]:
display(docs_df[docs_df['content'].str.len() != 0])
display(docs_df[(docs_df['content'].str.len() == 0) & (docs_df['title'] == '')])

,_id,title,url,published_at,summary,content,tags,article_type
0,633fdfd13ffae8229d05cb33,التربية: تحويل 42 مدرسة إلى نظام الفترتين واست...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 07:33:00,,[أكدت أمين عام وزارة التربية والتعليم للشؤون...,"[التربية والتعليم, وزارة التربية والتعليم]",News
1,633fdfd13ffae8229d05cb34,تكريما للمعلمين زيادة منح أبناء المعلمين 550 م...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-05 11:46:00,,[احتفلت وزارة التربية والتعليم بيوم المعلم ب...,"[مكرمة أبناء المعلمين, وزارة التربية والتعليم]",News
2,633fdfd13ffae8229d05cb35,الغاز الروسي يضع أوروبا خلال الشتاء المقبل أما...,https://husna.fm/%D9%85%D9%84%D9%81%D8%A7%D8%A...,2022-08-22 07:17:00,,[يشهد العالم أول أزمة طاقة عالمية حقيقية في ال...,"[الغاز الروسي, أوروبا, أزمة الطاقة]",News
3,633fdfd13ffae8229d05cb36,تفاصيل دوام المدارس بعد قرار تثبيت التوقيت الصيفي,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 05:29:00,,[كشفت أمين عام وزارة التربية والتعليم للشؤون ا...,"[التربية والتعليم, التوقيت الصيفي, المدارس]",News
4,633fdfd13ffae8229d05cb37,العمل: عطلة المولد النبوي الشريف تشمل القطاع ا...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 07:46:00,,[أكدت وزارة العمل أن العطل الرسمية تكون مأجو...,"[وزارة العمل, المولد النبوي]",News
...,...,...,...,...,...,...,...,...
6352,633ff10f322e7883822052b3,الديمقراطية والتقاء الخصوم,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-09-20 21:00:00,,[نتحدث هنا عن القوى المؤثرة على الأرض العربية ...,[الديموقراطية],Blog
6353,633ff10f322e7883822052b4,علموا أولادكم ألوان الطيف السبعة ومعانيها,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-09-21 12:18:00,,[سبحان من خلق الكون وأبدع في تصوريه، فجعل السم...,"[ألوان الطيف السبعة, الشذوذ الجنسي]",Blog
6356,633ff10f322e7883822052b7,تدين الخلاص الفردي وفرص النجاح,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-10-02 21:00:00,,[حالة من الصراع الصفري بين الأنظمة والإسلام ال...,"[الإسلام, الإسلام السياسي]",Blog
6358,633ff110322e7883822052b9,السر وراء الانخفاض الملحوظ في أسعار الأجهزة ال...,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-10-04 21:00:00,,[جدل يسود الشارع الأردني مجددا إثر التخفيضات ا...,[تخفيضات],Blog


,_id,title,url,published_at,summary,content,tags,article_type
1967,633fe2ce322e788382204192,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
1970,633fe2cf322e788382204195,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
1971,633fe2cf322e788382204196,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
1973,633fe2cf322e788382204198,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
1981,633fe2d0322e7883822041a0,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
...,...,...,...,...,...,...,...,...
6360,633ff111322e7883822052bb,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
6361,633ff111322e7883822052bc,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
6362,633ff113322e7883822052bd,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News
6363,633ff114322e7883822052be,,https://husna.fm/%D9%85%D9%84%D9%81-%D8%B4%D8%...,NaT,,[],[],News


In [31]:
docs_df_dropped = docs_df.drop(index=
                               docs_df[(docs_df['content'].str.len() == 0) & (docs_df['title'] == '')].index, axis=0)
docs_df_dropped = docs_df_dropped.reset_index(drop=True)
docs_df = docs_df_dropped

In [32]:
docs_df['text'] = docs_df['content'].apply(lambda x: " ".join(x))
docs_df['text'] 

0       أكدت أمين عام  وزارة  التربية والتعليم للشؤون ...
1       احتفلت  وزارة التربية والتعليم  بيوم المعلم بت...
2       يشهد العالم أول أزمة طاقة عالمية حقيقية في الت...
3       كشفت أمين عام وزارة التربية والتعليم للشؤون ال...
4       أكدت  وزارة العمل  أن العطل الرسمية تكون مأجور...
                              ...                        
5079    نتحدث هنا عن القوى المؤثرة على الأرض العربية ا...
5080    سبحان من خلق الكون وأبدع في تصوريه، فجعل السما...
5081    حالة من الصراع الصفري بين الأنظمة والإسلام الس...
5082    جدل يسود الشارع الأردني مجددا إثر التخفيضات ال...
5083    تميل درجات الحرارة في عطلة نهاية الأسبوع للارت...
Name: text, Length: 5084, dtype: object

#### 2.2 data cleaning

In [33]:
# punctuation symbols
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()
arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def clean_text(txt): 
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    txt = txt.translate(translator)
    
    # remove Tashkeel
    txt = re.sub(arabic_diacritics, '', txt)
    
    # remove longation
    txt = re.sub("[إأآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txt = re.sub("ؤ", "ء", txt)
    txt = re.sub("ئ", "ء", txt)
    txt = re.sub("ة", "ه", txt)
    txt = re.sub("گ", "ك", txt)
    
    # remove stopwords
    txt = ' '.join(word for word in txt.split() if word not in stop_words)
    
    return txt

In [34]:
docs_df['text_clean'] = docs_df['text'].apply(clean_text)
docs_df['summary_clean'] = docs_df['summary'].apply(clean_text)
docs_df['title_clean'] = docs_df['title'].apply(clean_text)
# + tags

display(docs_df)

,_id,title,url,published_at,summary,content,tags,article_type,text,text_clean,summary_clean,title_clean
0,633fdfd13ffae8229d05cb33,التربية: تحويل 42 مدرسة إلى نظام الفترتين واست...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 07:33:00,,[أكدت أمين عام وزارة التربية والتعليم للشؤون...,"[التربية والتعليم, وزارة التربية والتعليم]",News,أكدت أمين عام وزارة التربية والتعليم للشؤون ...,اكدت امين عام وزاره التربيه والتعليم للشءون ال...,,التربيه تحويل 42 مدرسه الي نظام الفترتين واستء...
1,633fdfd13ffae8229d05cb34,تكريما للمعلمين زيادة منح أبناء المعلمين 550 م...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-05 11:46:00,,[احتفلت وزارة التربية والتعليم بيوم المعلم ب...,"[مكرمة أبناء المعلمين, وزارة التربية والتعليم]",News,احتفلت وزارة التربية والتعليم بيوم المعلم بت...,احتفلت وزاره التربيه والتعليم بيوم المعلم بتكر...,,تكريما للمعلمين زياده منح ابناء المعلمين 550 م...
2,633fdfd13ffae8229d05cb35,الغاز الروسي يضع أوروبا خلال الشتاء المقبل أما...,https://husna.fm/%D9%85%D9%84%D9%81%D8%A7%D8%A...,2022-08-22 07:17:00,,[يشهد العالم أول أزمة طاقة عالمية حقيقية في ال...,"[الغاز الروسي, أوروبا, أزمة الطاقة]",News,يشهد العالم أول أزمة طاقة عالمية حقيقية في الت...,يشهد العالم اول ازمه طاقه عالميه حقيقيه التاري...,,الغاز الروسي يضع اوروبا خلال الشتاء المقبل اما...
3,633fdfd13ffae8229d05cb36,تفاصيل دوام المدارس بعد قرار تثبيت التوقيت الصيفي,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 05:29:00,,[كشفت أمين عام وزارة التربية والتعليم للشؤون ا...,"[التربية والتعليم, التوقيت الصيفي, المدارس]",News,كشفت أمين عام وزارة التربية والتعليم للشؤون ال...,كشفت امين عام وزاره التربيه والتعليم للشءون ال...,,تفاصيل دوام المدارس قرار تثبيت التوقيت الصيفي
4,633fdfd13ffae8229d05cb37,العمل: عطلة المولد النبوي الشريف تشمل القطاع ا...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2022-10-06 07:46:00,,[أكدت وزارة العمل أن العطل الرسمية تكون مأجو...,"[وزارة العمل, المولد النبوي]",News,أكدت وزارة العمل أن العطل الرسمية تكون مأجور...,اكدت وزاره العمل ان العطل الرسميه تكون ماجوره ...,,العمل عطله المولد النبوي الشريف تشمل القطاع الخاص
...,...,...,...,...,...,...,...,...,...,...,...,...
5079,633ff10f322e7883822052b3,الديمقراطية والتقاء الخصوم,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-09-20 21:00:00,,[نتحدث هنا عن القوى المؤثرة على الأرض العربية ...,[الديموقراطية],Blog,نتحدث هنا عن القوى المؤثرة على الأرض العربية ا...,نتحدث القوي المءثره علي الارض العربيه اصبحت سب...,,الديمقراطيه والتقاء الخصوم
5080,633ff10f322e7883822052b4,علموا أولادكم ألوان الطيف السبعة ومعانيها,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-09-21 12:18:00,,[سبحان من خلق الكون وأبدع في تصوريه، فجعل السم...,"[ألوان الطيف السبعة, الشذوذ الجنسي]",Blog,سبحان من خلق الكون وأبدع في تصوريه، فجعل السما...,خلق الكون وابدع تصوريه فجعل السماء درجات والار...,,علموا اولادكم الوان الطيف السبعه ومعانيها
5081,633ff10f322e7883822052b7,تدين الخلاص الفردي وفرص النجاح,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-10-02 21:00:00,,[حالة من الصراع الصفري بين الأنظمة والإسلام ال...,"[الإسلام, الإسلام السياسي]",Blog,حالة من الصراع الصفري بين الأنظمة والإسلام الس...,حاله الصراع الصفري الانظمه والاسلام السياسي تو...,,تدين الخلاص الفردي وفرص النجاح
5082,633ff110322e7883822052b9,السر وراء الانخفاض الملحوظ في أسعار الأجهزة ال...,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,2022-10-04 21:00:00,,[جدل يسود الشارع الأردني مجددا إثر التخفيضات ا...,[تخفيضات],Blog,جدل يسود الشارع الأردني مجددا إثر التخفيضات ال...,جدل يسود الشارع الاردني مجددا اثر التخفيضات ال...,,السر وراء الانخفاض الملحوظ اسعار الاجهزه الكهر...


#### 2.3 info and preprocessing

In [45]:
def show_info_text(df_col):
    print(f"-> Number of Documents: {docs_df.shape[0]}")
    print('-' * 50, end='\n\n')

    print('-> Documents - First 150 letters')
    print()
    for i, document_i in enumerate(docs_df['text_clean'][:20]):
        print(f"Document Number {i+1}: {document_i[:150]}..")
        print()

    print('-' * 50)
    
def data_preprocessing(df_col):
    # Instantiate a TfidfVectorizer object
    global vectorizer
    vectorizer = TfidfVectorizer()
    
    # It fits the data and transform it as a vector
    X = vectorizer.fit_transform(df_col)
    # Convert the X as transposed matrix
    X = X.T.toarray()
    # Create a DataFrame and set the vocabulary as the index
    df = pd.DataFrame(X, index=vectorizer.get_feature_names())
    return df

In [46]:
show_info_text(docs_df['text_clean'])
text_clean_enc_df = data_preprocessing(docs_df['text_clean'])
text_clean_enc_df 

-> Number of Documents: 5084
--------------------------------------------------

-> Documents - First 150 letters

Document Number 1: اكدت امين عام وزاره التربيه والتعليم للشءون الماليه والاداريه الدكتوره نجوي القبيلات ان التوسع دوام الطلبه مدارس وزاره التربيه علي نظام الفترتين جاء ب..

Document Number 2: احتفلت وزاره التربيه والتعليم بيوم المعلم بتكريمها نخبه المعلمات والمعلمين مختلف مديريات التربيه والتعليم بالمملكه خلال الحفل اقامته اليوم مدرسه كنجز ..

Document Number 3: يشهد العالم اول ازمه طاقه عالميه حقيقيه التاريخ تنامي المخاوف انقطاع امدادات الغاز الروسي وكما حذرت وكاله الطاقه الدوليه لعده اشهر فان الوضع محفوف بال..

Document Number 4: كشفت امين عام وزاره التربيه والتعليم للشءون الماليه والاداريه الدكتوره نجوي القبيلات بان مجلس الوزراء طلب منا السيناريوهات المقترحه حال تثبيت التوقيت ..

Document Number 5: اكدت وزاره العمل ان العطل الرسميه تكون ماجوره للعاملين مءسسات القطاع الخاص اذا عملوا ويستحقوا عنها اجرا اضافيا بواقع 150 اجرهم المعتاد وفقا لاحكام الم..

Document Number 6: و

C:\Users\modaj\anaconda3\envs\DataEngineering\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,5074,5075,5076,5077,5078,5079,5080,5081,5082,5083
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ደስ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ግድብ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ፈሶአል,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
全国初の再稼働,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
text_clean_enc_df.index[8000: 9000]

Index(['اسمده', 'اسمع', 'اسمعوني', 'اسمك', 'اسمنت', 'اسمنتي', 'اسمنتيه',
       'اسمنيه', 'اسمه', 'اسمها',
       ...
       'اغونا', 'اغير', 'اغيرهم', 'اف', 'اف35', 'افات', 'افاتار', 'افاجا',
       'افاد', 'افادات'],
      dtype='object', length=1000)

## 3- Calculating Similarities

In [39]:
def get_similar_articles(q, df, vectorizer):
    # Convert the query become a vector
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    
    # Calculate the similarity
    sim = {}
    for i in range(df.shape[1]):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
        if np.isnan(sim[i]):
            sim[i] = 0

    # Sort the values 
    sim_sorted = list(sim.items())
    return sim_sorted

In [43]:
# Add The Query
q1 = 'مولد النبي'

# Measures
time_measure = None
most_freq_measure = None  

start_time = time.time()
sorted_docs_with_scores = get_similar_articles(q1, text_clean_enc_df)  # call function
time_measure = (time.time() - start_time) * 10**3

sorted_docs_with_scores[:10]

C:\Users\modaj\AppData\Local\Temp\ipykernel_164320\3369844079.py:9: RuntimeWarning: invalid value encountered in double_scalars
  sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)


[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0)]

In [44]:
vocab_ = vectorizer.vocabulary_
print(f"number of unique words: {len(vocab_.keys())}")
most_freq_word = sorted(vocab_.items(), key=lambda x: x[1], reverse=True)[:1][0]
print('most frequent word is --> {} ({} times)'.format(most_freq_word[0], most_freq_word[1]))
score = len(vocab_.keys()) / most_freq_word[1]
print('Ratio: {:.3f}'.format(score))

most_freq_measure = most_freq_word[1]

number of unique words: 89124
most frequent word is --> 関西電力美浜3号機 (89123 times)
Ratio: 1.000


## 4- getting top documents

In [61]:
sorted_docs_with_scores = sorted(sorted_docs_with_scores, key=lambda x: x[1], reverse=True)
top_5_docs = np.array(sorted_docs_with_scores, dtype='int32')[:5, 0]
top_5_docs

array([1697, 2152, 2608, 2527, 2147])

In [62]:
# results 
print('time measure:', time_measure)
print('frequency measure:', most_freq_measure)
print('score %.3f' % score)

time measure: 648.0000019073486
frequency measure: 89123
score 1.000


## 5- Organizing Search Process

In [72]:
docs_df['text_clean'] = docs_df['text'].apply(clean_text)
docs_df['summary_clean'] = docs_df['summary'].apply(clean_text)
docs_df['title_clean'] = docs_df['title'].apply(clean_text)
# + tags

In [73]:
docs_df.head(1)

,title,url,published_at,crawled_at,summary,content,tags,article_type,text,text_clean,summary_clean,title_clean
0,الزراعة تدرس أفكارا لشركات خاصة لاستثمار مياه ...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,2021-09-01 06:55:00,2022-10-07 08:32:02.179500,كشف أمين عام وزارة الزراعة محمد الحياري لـ حسن...,[كما أكد الحياري أن التعليمات الجديدة التي ستط...,"[مياه الزيبار, وزارة الزراعة, زيت الزيتون]",News,كما أكد الحياري أن التعليمات الجديدة التي ستطب...,اكد الحياري ان التعليمات الجديده ستطبق اعتبارا...,كشف امين عام وزاره الزراعه محمد الحياري حسني ا...,الزراعه تدرس افكارا لشركات خاصه لاستثمار مياه ...


In [78]:
docs_df['tags']

0    [مياه الزيبار, وزارة الزراعة, زيت الزيتون]
1                            [السعودية, الأردن]
2                          [ذكرى المولد النبوي]
Name: tags, dtype: object

In [100]:
q_test = 'مياه الزراعة'
doc_ids = list(docs_df['tags'].index)
q_list = np.array(q_test.split(' '))
sim_score = list(np.zeros(docs_df.shape[0]))

for i, tag in enumerate(docs_df['tags']):
    print(tag)
    for str_tag in tag:
        q_list_map = np.vectorize(lambda x: x in str_tag)(q_list) 
        if True in q_list_map:
            sim_score[i] += 1
        print(q_list_map)
        
        
sim_non_sorted = list(zip(doc_ids, sim_score))
sim_sorted = sorted(sim_non_sorted, key=lambda x: x[1], reverse=True)

sim_sorted

['مياه الزيبار', 'وزارة الزراعة', 'زيت الزيتون']
[ True False]
[False  True]
[False False]
['السعودية', 'الأردن']
[False False]
[False False]
['ذكرى المولد النبوي']
[False False]


[(0, 2.0), (1, 0.0), (2, 0.0)]

In [188]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

cols_weights = [0.1, 0.5, 0.15, 0.2]
cols = ['tags', 'text_clean', 'summary_clean', 'title_clean']
def overall(q, df):
    # print("query:", q)
    
    # note: potential class vars
    overall_time = 0
    overall_score_measure = 0
    similarities_list = []
    
    # apply search over every col of interest
    for col in cols:
        if col != 'tags':
            # 1 data preprocessing(each column)
            text_clean_enc_df = data_preprocessing(docs_df[col])

            # 2- count similarities (each column)
            time_measure = None
            most_freq_measure = None  
            start_time = time.time()
            sorted_docs_with_scores_content = get_similar_articles(q, text_clean_enc_df)  # call function
            time_measure = (time.time() - start_time) * 10**3

            # 3- results
            global vectorizer
            vocab_ = vectorizer.vocabulary_
            # print(f"number of unique words: {len(vocab_.keys())}")
            most_freq_word = sorted(vocab_.items(), key=lambda x: x[1], reverse=True)[:1][0]
            # print('most frequent word is --> {} ({} times)'.format(most_freq_word[0], most_freq_word[1]))

            score = len(vocab_.keys()) / most_freq_word[1]
            # print('Ratio: {:.3f}'.format(score))
            # print()
            # print('time measure:', time_measure)
        else:
            time_measure = None
            most_freq_measure = None  
            start_time = time.time()
            
            doc_ids = list(docs_df['tags'].index)
            q_list = np.array(q.split(' '))
            sim_score = list(np.zeros(docs_df.shape[0]))

            for i, tag in enumerate(docs_df['tags']):
                # print(tag)
                for str_tag in tag:
                    q_list_map = np.vectorize(lambda x: x in str_tag)(q_list) 
                    if True in q_list_map:
                        sim_score[i] += 1
                    # print(q_list_map)

            sim_non_sorted = list(zip(doc_ids, sim_score))
            sorted_docs_with_scores_content = sim_non_sorted
            
            time_measure = (time.time() - start_time) * 10**3
            score = 0
            
        similarities_list.append(sorted_docs_with_scores_content)
        overall_time += time_measure
        overall_score_measure += (score/(len(cols) - 1))
        
        averaged_scores_ids = np.array(resulting_simalarities)[0, :, 0]
        averaged_scores = np.average(np.array(resulting_simalarities)[:, :, 1], axis=0, weights=cols_weights)
        similarities_scores = list(zip(averaged_scores_ids, averaged_scores))
        # print('--------------------')
    
    return similarities_scores, overall_time, overall_score_measure
    
    
q1 = 'مولد النبي'
resulting_simalarities, SE_time, SE_avg_score = overall(q1, docs_df)

print("-" * 25, 'FINAL', '-' * 25)
print('resulting simalarities:')
for rs in resulting_simalarities:
    print(rs)
print()
print('search engine time taken', SE_time, 'ms')
print('search engine average score', SE_avg_score, '(uniqueness/frequency)')
print("-" * 25, '-----', '-' * 25)

------------------------- FINAL -------------------------
resulting simalarities:
(0.0, 0.0)
(1.0, 0.0)
(2.0, 0.15821931459273364)

search engine time taken 2.998828887939453 ms
search engine average score 1.021043771043771 (uniqueness/frequency)
------------------------- ----- -------------------------


C:\Users\modaj\anaconda3\envs\DataEngineering\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\modaj\anaconda3\envs\DataEngineering\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\modaj\AppData\Local\Temp\ipykernel_143704\3369844079.py:9: RuntimeWarning: invalid value encountered in double_scalars
  sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
C:\Users\modaj\anaconda3\envs\DataEngineering\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; g

In [187]:
# TEST: COMPUTING AVERAGE SIMILARITY SCORES
# averaged_scores_ids = np.array(resulting_simalarities)[0, :, 0]
# display(averaged_scores_ids)

# display(np.array(resulting_simalarities)[:, :, 1])

# averaged_scores = np.average(np.array(resulting_simalarities)[:, :, 1], axis=0, weights=cols_weights)
# display(averaged_scores)

# list(zip(averaged_scores_ids, averaged_scores))

array([0., 1., 2.])

array([[0.       , 0.       , 1.       ],
       [0.       , 0.       , 0.1006167],
       [0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       ]])

array([0.        , 0.        , 0.15821931])

[(0.0, 0.0), (1.0, 0.0), (2.0, 0.15821931459273364)]